In [1]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [2]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [3]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [4]:
# Start SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
        master('local').\
        appName('Structured Stream').\
        getOrCreate()

In [5]:
# Define Schema de Dataframe
from pyspark.sql.types import *

esquema = StructType([
            StructField('person_ID',   StringType(), True),
            StructField('name',        StringType(), True),
            StructField('first',       StringType(), True),
            StructField('last',        StringType(), True),
            StructField('middle',      StringType(), True),
            StructField('email',       StringType(), True),
            StructField('phone',       StringType(), True),
            StructField('fax',         StringType(), True),
            StructField('title',       StringType(), True)])

In [6]:
# Cria Dataframe a partir de Fluxo Continuo de Dados
df = spark.\
            readStream.\
            schema(esquema).\
            option('header', False).\
            csv('Streaming')


In [7]:
# Print Schema
df.printSchema()

root
 |-- person_ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- middle: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- title: string (nullable = true)



In [8]:
# Confirma Objeto ser Streaming e Apresenta Descrição do Tipo de Objeto
print(df.isStreaming,'para Streaming - Tipo:',type(df))

True para Streaming - Tipo: <class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
# Gera novo Dataframe consultando todos os dados do Dataframe Original
df_results = df.select('*')

In [10]:
# Gera Objeto de Saida com Resultado
query = df_results \
        .writeStream \
        .outputMode('append') \
        .format('json') \
        .option('path', 'Results') \
        .option('checkpointLocation', 'Checkpoint') \
        .start()


In [ ]:
query.awaitTermination()

In [12]:
df2 = spark.read.json('/content/Results/part-00001.json')

In [13]:
df2.printSchema()

root
 |-- email: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- middle: string (nullable = true)
 |-- name: string (nullable = true)
 |-- person_ID: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- title: string (nullable = true)



In [14]:
df2.count()

40

In [15]:
df2.show(5)

+----------------+------------+-------+------+------+-------------------+---------+-----------------+-------------------+
|           email|         fax|  first|  last|middle|               name|person_ID|            phone|              title|
+----------------+------------+-------+------+------+-------------------+---------+-----------------+-------------------+
| BurksR@univ.edu|963.777.4065|Rosella| Burks|  null|    Burks, Rosella |     3130|     963.555.1253|         Professor |
| AvilaD@univ.edu|963.777.7914| Damien| Avila|  null|     Avila, Damien |     3297|     963.555.1352|         Professor |
| OlsenR@univ.edu|963.777.9262|  Robin| Olsen|  null|      Olsen, Robin |     3547|     963.555.1378|Assistant Professor|
|MoisesE@univ.edu|963.777.8264|  Edgar|Moises| Estes|Moises, Edgar Estes|     1538|963.555.2731x3565|          Professor|
| BrianH@univ.edu|963.777.7249|  Heath| Brian|Pruitt|Brian, Heath Pruitt|     2941|     963.555.2800| Associate Curator |
+----------------+------

In [ ]:
# Excluir Pasta com Conteudo, caso seja necessario para Reprocessar o Streaming
!rm -rf /content/Streaming
!rm -rf /content/Results
!rm -rf /content/Checkpoint

In [ ]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
spark.stop()